In [17]:
%load_ext autoreload
%autoreload 2
import pyec7

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [20]:
pyec7.bearing_resistance(width=1,log=True)

Geometry:
B = 1 m
L = 0 m
D = 0 m
a = 0 deg

Soil Properties:
gamma = 20 kN/m3
phi = 30 deg
c = 10 kPa

Loading:
V = None kN
HB = 0 kN
HL = 0 kN
H = 0 kN
MB = 0 kNm
ML = 0 kNm
q = 0 kPa

Effective Dimensions:
eB = 0 m
eL = 0 m
Be = 1 m
Le = 0 m
Ae = 1 m2

Bearing Capacity Factors:
Nq = 18.401122218708668
Nc = 30.139627791519086
Ng = 20.093085194346052

Inclination Factors:
bq = 1.0
bc = 1.0
bg = 1.0

Shape Factors:
sq = 1
sc = 1
sg = 1

Load Inclination Factors:
iq = 1
ic = 1
ig = 1

Results:
qu = 502.3271298586514 kPa
R = 502.3271298586514 kN


502.3271298586514

In [21]:
pyec7.earth_pressures(active=True,log=True)

Input parameters:
phi = 30 deg
c = -10 kPa
delta = 0 deg
beta = 0 deg
theta = 0 deg
q = 0 kPa
p = 0.0 kPa

Boundary conditions:
mw = 59.99999999999999 deg
v = 0.0 deg
mt = 59.99999999999999 deg

Coefficients:
Kn = 0.3333333333333333
Kq = 0.3333333333333333
Kc = 1.1547005383792517
Kg = 0.3333333333333333


(0.3333333333333333,
 0.3333333333333333,
 1.1547005383792517,
 0.3333333333333333)